<a href="https://colab.research.google.com/github/inthiyaz141176/Eyeball_position_detection/blob/master/autoencoder_rbf_cifar.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from keras.datasets import cifar10
import numpy as np

(x_train, _), (x_test, _) = cifar10.load_data()

# As before, let's make everything float and scale
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255
x_test /= 255

Using TensorFlow backend.


170500096/170498071 [==============================] - 34s 0us/step


In [2]:
from keras.layers import Input, Dense, Conv2D, MaxPooling2D, UpSampling2D,Flatten
from keras.models import Model
from keras import backend as K

def encoder(input_img):
  x = Conv2D(16, (3, 3), activation='relu', padding='same')(input_img)
  x = MaxPooling2D((2, 2), padding='same')(x)
  x = Conv2D(8, (3, 3), activation='relu', padding='same')(x)
  x = MaxPooling2D((2, 2), padding='same')(x)
  x = Conv2D(8, (3, 3), activation='relu', padding='same')(x)
  encoded = MaxPooling2D((2, 2), padding='same')(x)
  flatten=Flatten()(encoded)
  return encoded, flatten
# at this point the representation is (4, 4, 8) i.e. 128-dimensional

def decoder(input_img):
  encoded=encoder(input_img)
  x = UpSampling2D((2, 2))(encoded[0])
  x = Conv2D(8, (3, 3), activation='relu', padding='same')(x)
  x = UpSampling2D((2, 2))(x)
  x = Conv2D(8, (3, 3), activation='relu', padding='same')(x)
  x = UpSampling2D((2, 2))(x)
  x = Conv2D(16, (3, 3), activation='relu',padding='same')(x)
  decoded = Conv2D(3, (3, 3), activation='sigmoid', padding='same')(x)
  return decoded
input_img = Input(shape=x_train.shape[1:])
autoencoder = Model(input_img, decoder(input_img))
autoencoder.compile(loss='binary_crossentropy', optimizer='adadelta')
autoencoder.summary()

Instructions for updating:
Colocations handled automatically by placer.
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 32, 32, 3)         0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 32, 32, 16)        448       
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 16, 16, 16)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 16, 16, 8)         1160      
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 8, 8, 8)           0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 8, 8, 8)           584       
_________________________________________________________________
max_

In [3]:


autoencoder.fit(x_train, x_train,
                epochs=100,
                batch_size=32,
                shuffle=True,
                validation_data=(x_test, x_test))

Instructions for updating:
Use tf.cast instead.
Train on 50000 samples, validate on 10000 samples
Epoch 1/100
50000/50000 [==============================] - 17s 343us/step - loss: 0.6116 - val_loss: 0.5969
Epoch 2/100
50000/50000 [==============================] - 11s 223us/step - loss: 0.5947 - val_loss: 0.5963
Epoch 3/100
50000/50000 [==============================] - 11s 223us/step - loss: 0.5919 - val_loss: 0.5883
Epoch 4/100
50000/50000 [==============================] - 11s 223us/step - loss: 0.5876 - val_loss: 0.5852
Epoch 5/100
50000/50000 [==============================] - 12s 232us/step - loss: 0.5849 - val_loss: 0.5845
Epoch 6/100
50000/50000 [==============================] - 12s 239us/step - loss: 0.5838 - val_loss: 0.5874
Epoch 7/100
50000/50000 [==============================] - 11s 223us/step - loss: 0.5829 - val_loss: 0.5857
Epoch 8/100
50000/50000 [==============================] - 11s 222us/step - loss: 0.5823 - val_loss: 0.5908
Epoch 9/100
50000/50000 [=============

In [4]:
autoencoder.save_weights('auto_encoder_rbf.h5')
a=encoder(input_img)
encoder_model=Model(input_img,a[1])

encoder_model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 32, 32, 3)         0         
_________________________________________________________________
conv2d_8 (Conv2D)            (None, 32, 32, 16)        448       
_________________________________________________________________
max_pooling2d_4 (MaxPooling2 (None, 16, 16, 16)        0         
_________________________________________________________________
conv2d_9 (Conv2D)            (None, 16, 16, 8)         1160      
_________________________________________________________________
max_pooling2d_5 (MaxPooling2 (None, 8, 8, 8)           0         
_________________________________________________________________
conv2d_10 (Conv2D)           (None, 8, 8, 8)           584       
_________________________________________________________________
max_pooling2d_6 (MaxPooling2 (None, 4, 4, 8)           0         
__________

In [0]:
for l1,l2 in zip(encoder_model.layers[:7],autoencoder.layers[0:7]):
    l1.set_weights(l2.get_weights())

In [0]:
x_train=encoder_model.predict(x_train)
x_test=encoder_model.predict(x_test)

In [7]:
print(x_train.shape)
print(x_test.shape)

(50000, 128)
(10000, 128)


In [0]:
from sklearn.gaussian_process.kernels import PairwiseKernel
import keras
#from sklearn.metrics.pairwise import pairwise_kernels
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA
from sklearn.gaussian_process import GaussianProcessRegressor
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras.optimizers import RMSprop, Adam, Nadam

In [9]:
kmeans_model = KMeans(10)   #find 10 centers with kmeans on training samples
kmeans_model.fit(x_train)
centers = kmeans_model.cluster_centers_
x = kmeans_model.predict(kmeans_model.cluster_centers_)
x = keras.utils.to_categorical(x, 10)


y_trainn= kmeans_model.predict(x_train)
y_trainn=keras.utils.to_categorical(y_trainn,10)
y_testt=kmeans_model.predict(x_test)
y_testt=keras.utils.to_categorical(y_testt, 10)


#                       RBF LAYER

kernel = PairwiseKernel(metric='polynomial')   #GPR uses the kernel to define the covariance of the training sample
rbf_model = GaussianProcessRegressor(kernel=kernel).fit(centers, x)

temp1 = rbf_model.predict(x_train)
temp2 = rbf_model.predict(x_test)


#                      PERCEPTRONS LAYERS
batch_size = 128
epochs = 10

model = Sequential()
model.add(Dense(512, activation='relu', input_shape=(10,)))
model.add(Dropout(0.2))
model.add(Dense(10, activation='softmax'))

model.summary()
nadam=keras.optimizers.Nadam(lr=0.0005)
model.compile(loss='categorical_crossentropy',
              optimizer=nadam,
              metrics=['accuracy'])

#                      TRAINING THE MODEL
history = model.fit(temp1, y_trainn,
                    batch_size=batch_size,
                    epochs=epochs,
                    verbose=1,
                    validation_data=(temp2, y_testt))

#                      EVALUATE TRAINED MODEL
score = model.evaluate(temp2, y_testt, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])




Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 512)               5632      
_________________________________________________________________
dropout_1 (Dropout)          (None, 512)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 10)                5130      
Total params: 10,762
Trainable params: 10,762
Non-trainable params: 0
_________________________________________________________________
Train on 50000 samples, validate on 10000 samples
Epoch 1/10
50000/50000 [==============================] - 2s 43us/step - loss: 1.1652 - acc: 0.6886 - val_loss: 0.6185 - val_acc: 0.8191
Epoch 2/10
50000/50000 [==============================] - 1s 26us/step - loss: 0.4863 - acc: 0.8514 